In [13]:
import itertools as it
import collections as cc
import pandas as pd
import spacy as sp
import torch as pt

In [15]:
tags_ = pd.read_csv("./data/tags.csv")
tags_.head(10)

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn
5,90,branch
6,90,branching-and-merging
7,120,sql
8,120,asp.net
9,120,sitemap


In [20]:
tagCounter_ = cc.Counter(tags_.Tag)

In [22]:
tags_ = tags_[tags_.Tag.map(lambda x : tagCounter_[x]) > 500]

In [28]:
tags_ = tags_.set_index("Id")

In [45]:
questionsChunks_ = pd.read_csv("./data/questions4.csv", chunksize = 1000)


In [56]:
answers_ = pd.read_csv("./data/answer3.csv")

In [59]:
answers_ = answers_[["ParentId", "CleanBody"]]

In [67]:
len(tags_.index)

1212326

In [69]:
answers_ = answers_.set_index("ParentId").filter(tags_.index, axis = 0)

In [73]:
answers_.rename(columns = {"ParentId": "Id", "CleanBody": "AnswerBody"}, inplace = True)

In [54]:
tags_ = tags_.groupby("Id").aggregate(list)

In [79]:
questionsChunks_.get_chunk().set_index("Id").join(tags_, "Id", how = "inner").join(answers_, "Id", how = "left")

,Title,CleanBody,Tag,AnswerBody
Id,,,,
293300,"""Unknown column 'user_id' error in django view",I'm having an error where I am not sure what c...,"[python, django, django-models, model, view]",The field is the FK reference from to . It loo...
293380,Why Isn't My C Code Being Compiled To An EXE,I'm just starting out writing trying to write ...,"[c, visual-studio]",By default when you're creating a new C++ proj...
293500,How to deploy an RubyGem-based Server,We have built a custom socket server in ruby a...,"[ruby, deployment, rubygems]",You can install gems from the local filesystem...
293570,Can I use CSS in directory listing of Firefox?,"For example if I put ""C:\docs"" in address bar ...",[firefox],Firefox has a file userContent.css for each pr...
293590,Dynamically include .js files from Zend_Contro...,I came across the case where depending on the ...,[zend-framework],See the view helper headScript(). I'm just wri...
...,...,...,...,...
354110,"In Javascript, what is the difference between ...","Being fairly new to Javascript, I'm unable to ...","[javascript, string]",I think the main difference is that search acc...
354370,Adding a SOAP header to a SOAPpy request,Does anyone know how to do this? I need to add...,[python],"As the question is phrased, it's hard to guess..."
354400,Transitions and setting up Layers/Scenes in co...,I am looking to setup a transition between two...,"[objective-c, iphone, cocos2d-iphone]","For those who might care, here's what I ended ..."


In [87]:
for i, chunk_ in enumerate(pd.read_csv("./data/questions4.csv", chunksize = 50000)):
    chunk_ = chunk_.set_index("Id").join(tags_, "Id", how = "inner").join(answers_, "Id", how = "left")
    chunk_.to_csv("./data/tagedquestions_{0:03d}.csv".format(i))
    